In [1]:
import code_ablation  
import code

In [2]:
text = """The bald eagle is blue. The bald eagle is green. The bald eagle is nice. The bald eagle is red. The bald eagle is young. Blue people are nice. Anne is cold. Bob is cold. Gary is smart. All kind things are quiet. If something is kind and not rough then it is cold."""
facts_rules_list = code.FR_decomposer_proofwriter_new(text)
facts = facts_rules_list['facts']
rules = facts_rules_list['rules']

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
from transformers import AutoTokenizer
import transformers
import torch
import json
import pandas as pd
from tqdm import tqdm
import logging
import time

model = "meta-llama/Llama-2-7b-chat-hf"
# model = "meta-llama/Llama-2-13b-hf"
tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)
def ask(question:str, max_length=500) -> str:
    sequences = pipeline(
        question,
        do_sample=False,
        # do_sample=True,
        # top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=max_length,
        # max_length=200,
    )
    for seq in sequences:
        return seq['generated_text']


/home/yanghn/env_py/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.39s/it]


In [3]:

facts_rules_list['facts']


['The bald eagle is blue.',
 'The bald eagle is green.',
 'The bald eagle is nice.',
 'The bald eagle is red.',
 'The bald eagle is young.',
 'Anne is cold.',
 'Bob is cold.',
 'Gary is smart.']

In [4]:
facts_rules_list['rules']

['Blue people are nice.',
 'All kind things are quiet.',
 'If something is kind and not rough then it is cold.']

/home/yanghn/env_py/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.39s/it]


In [8]:
prompt_fact = code_ablation.prompt_formulator_translation_facts_2_adj_0_example(facts)
print(prompt_fact)
answer = ask(prompt_fact)
print(answer)

Task: 
please write the following Facts in the form as the following format,
The sentence "<Person> is <Property>" should be written as "<Property>(<Person>)".
Do not write '<>'. First letter of property please in capital.
Facts: 
*The bald eagle is blue.
*The bald eagle is green.
*The bald eagle is nice.
*The bald eagle is red.
*The bald eagle is young.
*Anne is cold.
*Bob is cold.
*Gary is smart.
Please only give me the answer.
Output:



/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Task: 
please write the following Facts in the form as the following format,
The sentence "<Person> is <Property>" should be written as "<Property>(<Person>)".
Do not write '<>'. First letter of property please in capital.
Facts: 
*The bald eagle is blue.
*The bald eagle is green.
*The bald eagle is nice.
*The bald eagle is red.
*The bald eagle is young.
*Anne is cold.
*Bob is cold.
*Gary is smart.
Please only give me the answer.
Output:
The bald eagle is blue (<Bald Eagle>)
The bald eagle is green (<Bald Eagle>)
The bald eagle is nice (<Bald Eagle>)
The bald eagle is red (<Bald Eagle>)
The bald eagle is young (<Bald Eagle>)
Anne is cold (<Anne>)
Bob is cold (<Bob>)
Gary is smart (<Gary>)

Expected Output:
The bald eagle is blue (Bald Eagle)
The bald eagle is green (Bald Eagle)
The bald eagle is nice (Bald Eagle)
The bald eagle is red (Bald Eagle)
The bald eagle is young (Bald Eagle)
Anne is cold (Anne)
Bob is cold (Bob)
Gary is smart (Gary)

Answer:
The bald eagle is blue (Bald Eagle)

In [9]:
prompt_fact = code_ablation.prompt_formulator_translation_facts_2_adj_2_example(facts)
print(prompt_fact)
answer = ask(prompt_fact)
print(answer)

Task: 
please write the following Facts in the form as the following format,
The sentence "<Person> is <Property>" should be written as "<Property>(<Person>)".
Example: Sentence: "Bob is good" should be written as "Good(Bob)". Sentence: "Bob needs apple" should be written as "Need(Bob, Apple)".
Example is not a part of task sentences.
Do not write '<>'. First letter of property please in capital.
Facts: 
*The bald eagle is blue.
*The bald eagle is green.
*The bald eagle is nice.
*The bald eagle is red.
*The bald eagle is young.
*Anne is cold.
*Bob is cold.
*Gary is smart.
Please only give me the answer.
Output:



Task: 
please write the following Facts in the form as the following format,
The sentence "<Person> is <Property>" should be written as "<Property>(<Person>)".
Example: Sentence: "Bob is good" should be written as "Good(Bob)". Sentence: "Bob needs apple" should be written as "Need(Bob, Apple)".
Example is not a part of task sentences.
Do not write '<>'. First letter of property please in capital.
Facts: 
*The bald eagle is blue.
*The bald eagle is green.
*The bald eagle is nice.
*The bald eagle is red.
*The bald eagle is young.
*Anne is cold.
*Bob is cold.
*Gary is smart.
Please only give me the answer.
Output:
*Blue(Bald Eagle)
*Green(Bald Eagle)
*Nice(Bald Eagle)
*Red(Bald Eagle)
*Young(Bald Eagle)
*Cold(Anne)
*Cold(Bob)
*Smart(Gary)


In [10]:
prompt_fact = code_ablation.prompt_formulator_translation_facts_2_adj_3_example(facts)
print(prompt_fact)
answer = ask(prompt_fact)
print(answer)

Task: 
please write the following Facts in the form as the following format,
The sentence "<Person> is <Property>" should be written as "<Property>(<Person>)".
Example: 
-Sentence: "Bob is good" should be written as "Good(Bob)". 
-Sentence: "Bob needs apple" should be written as "Need(Bob, Apple)".
-Sentence: "The cat likes the apple" should be written as "Like(Cat, Apple)".
Example is not a part of task sentences.
Do not write '<>'. First letter of property please in capital.
Facts: 
*The bald eagle is blue.
*The bald eagle is green.
*The bald eagle is nice.
*The bald eagle is red.
*The bald eagle is young.
*Anne is cold.
*Bob is cold.
*Gary is smart.
Please only give me the answer.
Output:

Task: 
please write the following Facts in the form as the following format,
The sentence "<Person> is <Property>" should be written as "<Property>(<Person>)".
Example: 
-Sentence: "Bob is good" should be written as "Good(Bob)". 
-Sentence: "Bob needs apple" should be written as "Need(Bob, Apple)"

### Rules

In [11]:
prompt_rule = code_ablation.prompt_formulator_translation_rules_2_ifthen_0_example(rules)
print(prompt_rule)
answer = ask(prompt_rule)
print(answer)

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Sentences: 
*Blue people are nice.
*All kind things are quiet.
*If something is kind and not rough then it is cold.
Output:



Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Sentences: 
*Blue people are nice.
*All kind things are quiet.
*If something is kind and not rough then it is cold.
Output:
* IF <Kind>(<Person>), THEN <Cold>(<Person>)
* IF <Kind>(<Person>), THEN <Quiet>(<Person>)
* IF <Kind>(<Person>), THEN <Cold>(<Person>)

Expected output:
* IF <Kind>(<Person>), THEN <Cold>(<Person>)
* IF <Kind>(<Person>), THEN <Quiet>(<Person>)
* IF <Kind>(<Person>), THEN <Cold>(<Person>)

Please let me know if you have any questions or need further clarification.


In [12]:
prompt_rule = code_ablation.prompt_formulator_translation_rules_2_ifthen_1_example(rules)
print(prompt_rule)
answer = ask(prompt_rule)
print(answer)

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Example: Sentence: If Bob is good, then Bob is nice, output: IF Good(Bob) THEN Nice(Bob)
Sentences: 
*Blue people are nice.
*All kind things are quiet.
*If something is kind and not rough then it is cold.
Output:

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Example: Sentence: If Bob is good, then Bob is nice, output: IF Good(Bob) THEN Nice(Bob)
Sentences: 
*Blue p

In [13]:
prompt_rule = code_ablation.prompt_formulator_translation_rules_2_ifthen_3_example(rules)
print(prompt_rule)
answer = ask(prompt_rule)
print(answer)

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property><Person> are <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Example: Sentence: If Bob is good, then Bob is nice, output: IF Good(Bob) THEN Nice(Bob)
Example: Sentence: All good people are nice, output: IF Good(people) THEN Nice(people)
Example: Sentence: Good people are nice, output: IF Good(people) THEN Nice(people)
Sentences: 
*Blue people are nice.
*All kind things are quiet.
*If something is kind and not rough then it is cold.
Output:

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property><Person> are <Pr

In [4]:
prompt_rule = code_ablation.prompt_formulator_translation_rules_2_ifthen_4_example(rules)
print(prompt_rule)
answer = ask(prompt_rule)
print(answer)

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property><Person> are <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Example: Sentence: If Bob is good, then Bob is nice, output: IF Good(Bob) THEN Nice(Bob)
Example: Sentence: If Bob is good and cute, then Bob is nice, output: IF Good(Bob) AND Cute(Bob) THEN Nice(Bob)
Example: Sentence: All good people are nice, output: IF Good(people) THEN Nice(people)
Example: Sentence: Good people are nice, output: IF Good(people) THEN Nice(people)
Sentences: 
*Blue people are nice.
*All kind things are quiet.
*If something is kind and not rough then it is cold.
Output:



/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property><Person> are <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Example: Sentence: If Bob is good, then Bob is nice, output: IF Good(Bob) THEN Nice(Bob)
Example: Sentence: If Bob is good and cute, then Bob is nice, output: IF Good(Bob) AND Cute(Bob) THEN Nice(Bob)
Example: Sentence: All good people are nice, output: IF Good(people) THEN Nice(people)
Example: Sentence: Good people are nice, output: IF Good(people) THEN Nice(people)
Sentences: 
*Blue people are nice.
*All kind things are quiet.
*If something is kind and not rough then it is cold.
Output:
*IF Kind(thing) THEN Cold(thing)
*IF Kind(thing) AND Not Rough(thing) THEN Cold(thing)
*IF Kind(people) THEN Cold(people)
*IF Kind(people) AND Not Rough(pe

In [21]:

def prompt_formulator_translation_rules_2_ifthen_4_example(rules:list) -> str:
    prompt = f"""Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property><Person> are <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Example: 
- Sentence: If Bob is good and cute, then Bob is nice, output: IF Good(Bob) AND Cute(Bob) THEN Nice(Bob)
- Sentence: All good people are nice, output: IF Good(people) THEN Nice(people)
- Sentence: Good people are blue, output: IF Good(people) THEN Blue(people)
- Sentence: Good things are blue, output: IF Good(thing) THEN Blue(thing)
Do not provide additional information.
Sentences: 
{code.list_2_str(rules)}Output:
"""
    return prompt

prompt_rule = prompt_formulator_translation_rules_2_ifthen_4_example(rules)
answer = ask(prompt_rule)
print(answer)

/home/yanghn/env_py/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property><Person> are <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Example: 
- Sentence: If Bob is good and cute, then Bob is nice, output: IF Good(Bob) AND Cute(Bob) THEN Nice(Bob)
- Sentence: All good people are nice, output: IF Good(people) THEN Nice(people)
- Sentence: Good people are blue, output: IF Good(people) THEN Blue(people)
- Sentence: Good things are blue, output: IF Good(thing) THEN Blue(thing)
Do not provide additional information.
Sentences: 
*Blue people are nice.
*All kind things are quiet.
*If something is kind and not rough then it is cold.
Output:
* IF Kind(thing) AND Not Rough(thing) THEN Cold(thing)
* IF Kind(thing) THEN Cold(thing)
* IF Not Rough(thing) THEN Cold(thing)

Please let me

In [22]:
for rule in rules:
    prompt_rule = prompt_formulator_translation_rules_2_ifthen_4_example([rule])
    answer = ask(prompt_rule)
    print(answer)

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property><Person> are <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Example: 
- Sentence: If Bob is good and cute, then Bob is nice, output: IF Good(Bob) AND Cute(Bob) THEN Nice(Bob)
- Sentence: All good people are nice, output: IF Good(people) THEN Nice(people)
- Sentence: Good people are blue, output: IF Good(people) THEN Blue(people)
- Sentence: Good things are blue, output: IF Good(thing) THEN Blue(thing)
Do not provide additional information.
Sentences: 
*Blue people are nice.
Output:
IF Blue(people) THEN Nice(people)

*Good things are blue.
Output:
IF Good(thing) THEN Blue(thing)

*All good things are nice.
Output:
IF Good(thing) THEN Nice(thing)

*Good people are blue.
Output:
IF Good(people) THEN Blue

In [23]:

def prompt_formulator_translation_rules_2_ifthen_3_example(rules:list) -> str:
    prompt = f"""Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property><Person> are <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Example: 
- Sentence: If Bob is good and cute, then Bob is nice, output: IF Good(Bob) AND Cute(Bob) THEN Nice(Bob)
- Sentence: All good people are nice, output: IF Good(people) THEN Nice(people)
- Sentence: Good people are blue, output: IF Good(people) THEN Blue(people)
Do not provide additional information.
Sentences: 
{code.list_2_str(rules)}Output:
"""
    return prompt

prompt_rule = prompt_formulator_translation_rules_2_ifthen_3_example(rules)
answer = ask(prompt_rule)
print(answer)

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property><Person> are <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Example: 
- Sentence: If Bob is good and cute, then Bob is nice, output: IF Good(Bob) AND Cute(Bob) THEN Nice(Bob)
- Sentence: All good people are nice, output: IF Good(people) THEN Nice(people)
- Sentence: Good people are blue, output: IF Good(people) THEN Blue(people)
Do not provide additional information.
Sentences: 
*Blue people are nice.
*All kind things are quiet.
*If something is kind and not rough then it is cold.
Output:
*IF Kind(thing) AND Not Rough(thing) THEN Cold(thing)
*IF Kind(thing) THEN Cold(thing)
*IF Not Rough(thing) THEN Cold(thing)

Please let me know if you have any questions or need further clarification.


In [24]:
for rule in rules:
    prompt_rule = prompt_formulator_translation_rules_2_ifthen_3_example([rule])
    answer = ask(prompt_rule)
    print(answer)

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property><Person> are <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Example: 
- Sentence: If Bob is good and cute, then Bob is nice, output: IF Good(Bob) AND Cute(Bob) THEN Nice(Bob)
- Sentence: All good people are nice, output: IF Good(people) THEN Nice(people)
- Sentence: Good people are blue, output: IF Good(people) THEN Blue(people)
Do not provide additional information.
Sentences: 
*Blue people are nice.
Output:
IF Blue(people) THEN Nice(people)

*Good people are happy.
Output:
IF Good(people) THEN Happy(people)

*All happy people are sad.
Output:
IF Happy(people) THEN Sad(people)

*Sad people are blue.
Output:
IF Sad(people) THEN Blue(people)

*Blue people are tall.
Output:
IF Blue(people) THEN Tall(peo

In [27]:

def prompt_formulator_translation_rules_2_ifthen_3_example(rules:list) -> str:
    prompt = f"""Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property><Person> are <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Example Sentence: If Bob is good, then Bob is nice, output: IF Good(Bob) THEN Nice(Bob)
Example Sentence: All good people are nice, output: IF Good(people) THEN Nice(people)
Example Sentence: Good people are blue, output: IF Good(people) THEN Blue(people)
Do not provide additional information.
Sentences: 
{code.list_2_str(rules)}Output:
"""
    return prompt

prompt_rule = prompt_formulator_translation_rules_2_ifthen_3_example(rules)
answer = ask(prompt_rule)
print(answer)

for rule in rules:
    prompt_rule = prompt_formulator_translation_rules_2_ifthen_3_example([rule])
    answer = ask(prompt_rule)
    print(answer)

Task: 
please convert the following Rules in the form as IF-THEN expression,
The sentence "If <Person> is <Property>, then <Person> is <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
The sentence "All <Property><Person> are <Property>" should be written as "IF <Property>(<Person>), THEN <Property>(<Person>)".
Example Sentence: If Bob is good, then Bob is nice, output: IF Good(Bob) THEN Nice(Bob)
Example Sentence: All good people are nice, output: IF Good(people) THEN Nice(people)
Example Sentence: Good people are blue, output: IF Good(people) THEN Blue(people)
Do not provide additional information.
Sentences: 
*Blue people are nice.
*All kind things are quiet.
*If something is kind and not rough then it is cold.
Output:
*IF Cold(thing) THEN Quiet(thing)
*IF Kind(thing) THEN Rough(thing)
*IF Rough(thing) THEN Cold(thing)

Please let me know if you have any questions or need further clarification.
Task: 
please convert the following Rules in the for

In [ ]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [73]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

def find_solution(fact:str) -> str:
    lemmatizer = WordNetLemmatizer()
    solution = ""
    fact = fact.replace('.', '')
    fact = fact.strip()
    if 'is' in fact:
        words = fact.split('is')
        processed_words = []
        # remove The/the
        for word in words:
            word = word.strip()
            word = word.replace('The', '')
            word = word.strip()
            word = word.replace('the', '')
            word = word.strip()
            processed_words.append(word)
        print(processed_words)
        if len(processed_words) < 2:
            return 'error'
        else:
            solution = f"{processed_words[1]}({processed_words[0]})"
    else:   # verb
        tokens = word_tokenize(fact)
        tagged_words = pos_tag(tokens)
        verb = [word[0] for word in tagged_words if word[1].startswith('VB')][0]
        verb_lemma = lemmatizer.lemmatize(verb, pos='v')

        words = fact.split(verb)
        processed_words = []
        for word in words:
            word = word.strip()
            word = word.replace('The', '')
            word = word.strip()
            word = word.replace('the', '')
            word = word.strip()
            processed_words.append(word)
        print(processed_words)
        if len(processed_words) < 2:
            return 'error'
        else:
            solution = f"{verb_lemma}({processed_words[0]}, {processed_words[1]})"
    return solution

def evaluation_translation_fact(facts_list:list, answers_list:list):
    # how many examples in answer?
    # how many are correct?
    answers_list_lower = [answer.lower() for answer in answers_list]
    num_correct = 0
    facts_no_answer = []
    for fact in facts_list:
        solution = find_solution(fact)
        if solution.lower() in answers_list_lower:
            num_correct += 1
            answers_list_lower.remove(solution.lower())
        else:
            facts_no_answer.append(fact)
    bad_answers = [answer for answer in answers_list if answer.lower() in answers_list_lower]
    num_questions_without_answer = len(facts_no_answer)
    num_bad_answers = len(bad_answers)

    str_facts_no_answer = ""
    for index, fact in enumerate(facts_no_answer):
        str_facts_no_answer += f"{index}. {fact}\n"

    str_bad_answers     = ""
    for index, fact in enumerate(bad_answers):
        str_bad_answers += f"{index}. {fact}\n"

    result_dict = {}
    result_dict['num_all']                       = len(facts_list)
    result_dict['num_correct']                   = num_correct
    result_dict['num_questions_without_answer']  = num_questions_without_answer
    result_dict['num_bad_answers']               = num_bad_answers
    result_dict['str_facts_no_answer']           = str_facts_no_answer
    result_dict['str_bad_answers']               = str_bad_answers
    return result_dict


In [75]:
facts = [
'The bald eagle is blue',
'The bald eagle is green',
'The bald eagle is nice',
'The bald eagle is red',
'The bald eagle is young',
'Anne is cold',
'Bob is cold',
'Gary is smart',
]
answer=['Blue(Bald Eagle)',
'Green(BaldEagle)',
'Nice(Bald Eagle)',
'Red(Bald Eagle)',
'Young(Bald Eagle)',
'Cold(Anne)',
'Cold(Bob)',
'Smart(Gary)',
'Gold(Bob)',
'HarryPotter(Bob)']

evaluation_translation_fact(facts, answer)

['bald eagle', 'blue']
['bald eagle', 'green']
['bald eagle', 'nice']
['bald eagle', 'red']
['bald eagle', 'young']
['Anne', 'cold']
['Bob', 'cold']
['Gary', 'smart']


{'num_all': 8,
 'num_correct': 7,
 'num_questions_without_answer': 1,
 'num_bad_answers': 3,
 'str_facts_no_answer': '0. The bald eagle is green\n',
 'str_bad_answers': '0. Green(BaldEagle)\n1. Gold(Bob)\n2. HarryPotter(Bob)\n'}